# 4. ニューラルネットワークの学習

Let's study neural network! ではなくニューラルネットワークにデータを与えて学習させるほう。

## 4.1関連：特徴量という考え方

[特徴量とは、一言で表すと、分析対象データの中の、予測の手掛かりとなる変数のことです。](https://www.sbbit.jp/article/cont1/76066)  
データの何を使って機械学習を行うか、と学習するためにデータを加工していく作業をfeature engineeringともいう。例えばデータセットの中に変数A,変数B,変数Cがあるとして、全てを機械学習アルゴリズムにかけるか、あるいはAとBだけというように一部だけ使うか、あるいは新たな変数を作るか（A＋BーC＝D的な）といった選択を行い、より精度の良いモデルを作ること。  
  
一般的な機械学習はデータ→特徴量を決める→形の変わったデータ→機械学習の流れ。  
  
一方でディープラーニングではその特徴量エンジニアリングがいらない。データをそのまま入力すると勝手に特徴量を見つける。すごい。